In [ ]:
import subprocess, textwrap, operator
from langgraph.graph import END, START, StateGraph
from langgraph.types import Send
from langchain.chat_models import init_chat_model
from typing import TypedDict
from openai import OpenAI
from typing_extensions import Annotated

llm = init_chat_model("openai:gpt-5-nano")


class State(TypedDict):
    video_file: str
    audio_file: str
    transcription: str
    summaries: Annotated[list[str], operator.add]

In [ ]:
def extract_audio(state: State):
    output_file = state["video_file"].replace("mp4", "mp3")
    command = [
        "ffmpeg",
        "-i",
        state["video_file"],
        "-filter:a",
        "atempo=2.0",
        "-y",
        output_file
    ]
    subprocess.run(command)
    return {
        "audio_file": output_file
    }

def transcribe_audio(state: State):
    client = OpenAI()
    with open(state["audio_file"], "rb") as audio_file:
        transcription = client.audio.transcriptions.create(
            model="whisper-1",
            response_format="text",
            file=audio_file,
            # language="en",
            # prompt="to give the model some hints for audio file"
        )
        return {
            "transcription": transcription
        }

def dispatch_summarizers(state: State):
    transcription = state["transcription"]
    chunks = []
    for i, chunk in enumerate(textwrap.wrap(transcription, 500)):
        chunks.append({"id": i + 1, "chunk": chunk})
    return [Send("summarize_chunk", chunk) for chunk in chunks]

def summarize_chunk(chunk_item):
    chunk_id = chunk_item["id"]
    chunk = chunk_item["chunk"]
    # print(f"Summarizing chunk id: {chunk_id} chunk: {chunk[:100]}\n\n====\n\n")

    response = llm.invoke(
        f"""
        Please summarize the following text.

        Text: {chunk}
        """
    )
    summary = f"[Chunk {chunk_id}] {response.content}"
    return {
        "summaries": [summary]
    }



In [16]:
graph_builder = StateGraph(State)

graph_builder.add_node("extract_audio", extract_audio)
graph_builder.add_node("transcribe_audio", transcribe_audio)
graph_builder.add_node("summarize_chunk", summarize_chunk)

graph_builder.add_edge(START, "extract_audio")
graph_builder.add_edge("extract_audio", "transcribe_audio")
graph_builder.add_conditional_edges("transcribe_audio", dispatch_summarizers, ["summarize_chunk"])
graph_builder.add_edge("summarize_chunk", END)

graph = graph_builder.compile()

In [17]:
graph.invoke({"video_file": "interview.mp4"})

Summarizing chunk id: 1 chunk: 서울 고려대학교 미역하에서 시작한 천 원짜리 영철 버거 핫도그 빵 사이에 고기 볶음과 양배추 소스 등을 넣은 버거는 큰 인기를 끌며 고려대의 명물이 됐습니다. 2000년 영철 버거

====


Summarizing chunk id: 2 chunk: 13일 18살의 나이로 세상을 떠났습니다. 빈손은 이 씨의 평생일도 고려대학교 안암병원 장례식장에 마련됐습니다. 외롭고 방황하던 청춘의 허기진 마음들을 이불처럼 덮어주셨던 사장님의

====




{'video_file': 'interview.mp4',
 'audio_file': 'interview.mp3',
 'transcription': '서울 고려대학교 미역하에서 시작한 천 원짜리 영철 버거 핫도그 빵 사이에 고기 볶음과 양배추 소스 등을 넣은 버거는 큰 인기를 끌며 고려대의 명물이 됐습니다. 2000년 영철 버거를 상업한 이영철 씨는 2004년부터 고려대학교에 매년 2천만 원을 기부했습니다. 학생들의 주머니 사정을 생각해 적자에도 천 원의 약속을 지켰습니다. 2015년엔 결국 경영난으로 폐업했는데 고려대 동문들이 나서 가게를 일으켜 세우며 화제가 되기도 했습니다. 10년 전 JTBC와의 인터뷰에서는 재개업을 앞둤던 이 씨의 두려움과 설렘이 고스란히 담겨 있었습니다. 기분이야 뭐 정말 새롭죠. 많이 걱정도 되고 두렵기도 하고 고맙기도 하고 앞으로 성장하고 있으면 어떻게 하실 거예요? 소심으로 돌아가서 열심히 일하고 또 학생들하고 많이 소통하고 하려고 합니다. 이 씨는 인터뷰 내내 미소를 지었습니다. 너무너무 감사하고 정말 그 고마움 잊지 않고 내가 살아가는 모습으로 보답하려고 해요. 암투병 중이던 이 씨는 지난 13일 18살의 나이로 세상을 떠났습니다. 빈손은 이 씨의 평생일도 고려대학교 안암병원 장례식장에 마련됐습니다. 외롭고 방황하던 청춘의 허기진 마음들을 이불처럼 덮어주셨던 사장님의 삶을 생기겠다 웃음이 눈가에 발자국 남는 사람이 되어 기억하겠다. 온라인 후보에는 1,000개 넘는 추모와 추억의 메시지가 속속 담겨지고 있습니다. 이 씨의 발의는 내일 이뤄집니다.\n'}